In [1081]:
import pandas as pd
import sys
sys.path[0] = "c:/Users/Robert/Documents/Projekte/dev/bettingTool/"


In [1082]:
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score, roc_auc_score,
                             roc_curve)
import numpy as np
import shap
import pandas as pd
import pickle
from src.utils import globals as GLOBALS
pd.set_option('display.max_columns', 500)

def load_team_opponent(filename_main:str):
    df = pickle.load(open(filename_main, 'rb'))
    mapping = pd.read_csv(GLOBALS.SETTINGS_FOLDER+'mapping_team_opponent.csv',"\t")


    map_home = dict()
    map_away=dict()
    for i in mapping.columns:
        if mapping[i][0]!="x":
            map_home[i] = mapping[i][0]
        if mapping[i][1]!="x":
            map_away[i] = mapping[i][1]

    df_home = df[[i for i in df.columns if i in  map_home]].rename(map_home,axis=1)
    df_home['atHome']=True

    df_away = df[[i for i in df.columns if i in  map_away]].rename(map_away,axis=1)
    df_away['atHome']=False


    df = pd.concat([df_home,df_away],axis=0)
    df = split_date(df = df)
    df['Date'] = pd.to_datetime(df[['year', 'month', 'day']])
    df.index =  df.Date.astype(str)+'_' + df.Team+'_' + df.Opponent
    return df

def split_date(df):
    if 'Date' in df.columns:
        new_date = (df["Date"].str.split("/",expand=True))
        new_date.columns = ["day","month","year"]
        new_date = new_date.astype('int')

        new_date.year[new_date.year < 2000] = new_date.year[new_date.year < 2000]+2000

        df = pd.concat([df,new_date],axis=1)
    else:
        print('Keine Spalte "Date" in Spalten vorhanden')
    return df

def set_dummies_div(df, cat, divs=[]):
    if cat in df.columns:
        if divs == []:
            divs = list(set(df[cat]))
        for d in divs:
            df[d] = [1 if ele == d else 0 for ele in df[cat]]
        df = df.drop(cat, axis=1)
    return df

def get_odd_pred(bet,df):
    odd_team = df[bet+"_Team"]
    odd_opponent = df[bet+"_Opponent"]
    odd_draw = df[bet+"_Draw"]
    sum_odds = odd_team + odd_opponent + odd_draw
    odd_pred = pd.DataFrame(index=df.index)
    odd_pred[bet+"_Team_odd_pred"]=odd_team/sum_odds
    odd_pred[bet+"_opponent_odd_pred"]=odd_opponent/sum_odds
    odd_pred[bet+"_Team_draw_pred"]=odd_draw/sum_odds
    
    return odd_pred
def evaluate_model(model, y_test, X_test, show=False,threshold=''):
    if threshold == '':
        y_pred = model.predict(X_test)
    else:
       y_pred = model.predict_proba(X_test)[:,1]>threshold

    modellguete = dict()
    modellguete["roc_auc_score"] = np.round(roc_auc_score(y_test, y_pred,), 3)
    modellguete["f1_score"] = np.round(f1_score(y_test, y_pred), 3)
    modellguete["accuracy_score"] = np.round(accuracy_score(y_test, y_pred), 3)
    modellguete["precision_score"] = np.round(
        precision_score(y_test, y_pred), 3)
    modellguete["recall_score"] = np.round(recall_score(y_test, y_pred), 3)

    print('AUC Score : ', modellguete["roc_auc_score"])
    print('F1 Score : ', modellguete["f1_score"])
    print('Accuracy Score : ', modellguete["accuracy_score"])
    print('Precision Score : ', modellguete["precision_score"])
    print('Recall Score : ', modellguete["recall_score"])

    # Model Accuracy, how often is the classifier correct?
    print('Confusion Matrix : /n' +
          str(np.round(confusion_matrix(y_test, y_pred), 3)))
    if True:
        try:
            print('-'*100)
            print('Feature Importance',)
            feature_imp = pd.Series(
                model.feature_importances_, index=X_test.columns).sort_values(ascending=False)[0:15]
            feature_imp.to_json()
            print('Important')
            print(feature_imp)
            feature_not_imp = pd.Series(
                model.feature_importances_, index=X_test.columns).sort_values(ascending=True)[0:15]
            print('Not Important')
            print(feature_not_imp)
        except:
            print('Fehler bei Feature Importance')
            pass

    if False:
        try:
            print('-'*100)
            print('Shap Values',)
            rf_shap_values = shap.TreeExplainer(model).shap_values(
                X_test.sample(100, random_state=42))
            shap_df = pd.DataFrame(rf_shap_values[0], columns=X_test.columns)
            print(np.abs(shap_df).mean().sort_values(ascending=False))
        except:
            pass

        try:
            X_test.corr().round(2)[X_test.corr().round(2) > 0.3].to_csv(
                'C:/Users/Robert/Documents/corr.csv', sep=';', decimal=',', index=True)
        except:
            pass

    print('-'*100)
    return modellguete
def get_bookmaker(bm:str,df):
    return [i for i in df.columns if bm in i]
def derived_odds(sight:str,df:pd.DataFrame,odds:list):
    return_df = pd.DataFrame()
    return_df[f'Max_{sight}'] = df[odds].apply(max,axis=1)
    return_df[f'Min_{sight}'] = df[odds].apply(min,axis=1)
    return_df[f'Avg_{sight}'] = df[odds].apply(np.average,axis=1)

    return_df[f'Span_{sight}'] = return_df[f'Max_{sight}']-return_df[f'Min_{sight}'] 
    return_df[f'Ratio_{sight}'] = return_df[f'Max_{sight}']/return_df[f'Min_{sight}'] 
    return return_df


# Lade Daten ein

## Team Opponent

In [1083]:
df = load_team_opponent(filename_main = GLOBALS.DATA_FOLDER+'main_leagues_database_raw.sav')

In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only


# Erstelle Zielvariable

In [1084]:
df['Target'] = df.FTG_Team>df.FTG_Opponent
df['Target'].value_counts()

False    93004
True     53862
Name: Target, dtype: int64

# Features entwicklen

In [1085]:
bookmaker_attributes = get_bookmaker(bm = 'BW',df = df)+get_bookmaker(bm='B365',df = df)+get_bookmaker(bm='IW',df = df)

In [1121]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

class addDericedBookmakerPred(BaseEstimator, TransformerMixin):
    def __init__(self, bookmaker):
        self.bookmaker = bookmaker
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        odds_pred = get_odd_pred(bet=self.bookmaker,df=X)
        X_prep = X.join(odds_pred)
        return X_prep

class addDericedBookmakerOdds(BaseEstimator, TransformerMixin):
    def __init__(self, bookmakers):
        bookmaker_attributes = []
        for bm in bookmakers:
            bookmaker_attributes += get_bookmaker(bm = bm,df = df)
        
        self.bookmaker_attributes = bookmaker_attributes
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        bet_on_team = get_bookmaker(bm='Team',df=df[bookmaker_attributes])
        odds_pred = derived_odds(sight = 'Team',df=df, odds=bet_on_team)
        X_prep = X.join(odds_pred)
        return X_prep

In [1122]:
add_odds = addDericedBookmakerOdds(bookmakers=['B365'])
addDericedBookmaker = Pipeline(
    steps=[
        ('B365',addDericedBookmakerOdds(bookmakery='B365'))
        ])
addDericedBookmaker.fit(df)
addDericedBookmaker.transform(df)

TypeError: __init__() got an unexpected keyword argument 'bookmaker'

In [1118]:
add_odds = addDericedBookmakerPred(bookmaker='B365')
addDericedBookmaker = Pipeline(
    steps=[
        ('B365',addDericedBookmakerPred(bookmaker='B365')),
        ('BW',addDericedBookmakerPred(bookmaker='BW')),
        ('IW',addDericedBookmakerPred(bookmaker='IW')),

        ])
addDericedBookmaker.fit(df)
addDericedBookmaker.transform(df)

,Avg_Opponent,Avg_Draw,Avg_Team,Opponent,B365_Opponent,B365_Draw,B365_Team,BW_Opponent,BW_Draw,BW_Team,Date,Div,FTG_Opponent,FTG_Team,R,Team,IW_Opponent,IW_Draw,IW_Team,Max_Opponent,Max_Draw,Max_Team,VC_Opponent,VC_Draw,VC_Team,WH_Opponent,WH_Draw,WH_Team,atHome,day,month,year,Target,B365_Team_odd_pred,B365_opponent_odd_pred,B365_Team_draw_pred,BW_Team_odd_pred,BW_opponent_odd_pred,BW_Team_draw_pred,IW_Team_odd_pred,IW_opponent_odd_pred,IW_Team_draw_pred
2022-08-05_Istanbulspor_Trabzonspor,1.71,3.80,4.66,Trabzonspor,1.61,3.75,4.70,1.70,3.75,4.60,2022-08-05,T1,2,0,A,Istanbulspor,1.70,3.85,4.80,1.77,4.10,5.00,1.73,3.8,4.75,1.67,3.80,4.75,True,5,8,2022,False,0.467197,0.160040,0.372763,0.457711,0.169154,0.373134,0.463768,0.164251,0.371981
2022-08-06_Sivasspor_Gaziantep,3.56,3.56,2.00,Gaziantep,3.30,3.50,1.95,3.50,3.50,2.00,2022-08-06,T1,1,1,D,Sivasspor,3.50,3.55,2.05,3.83,3.72,2.09,3.60,3.5,2.05,3.60,3.40,2.00,True,6,8,2022,False,0.222857,0.377143,0.400000,0.222222,0.388889,0.388889,0.225275,0.384615,0.390110
2022-08-06_Besiktas_Kayserispor,6.04,4.39,1.50,Kayserispor,5.25,4.33,1.50,5.75,4.20,1.51,2022-08-06,T1,0,1,H,Besiktas,6.00,4.30,1.53,6.50,4.70,1.55,6.50,4.2,1.50,6.50,4.20,1.47,True,6,8,2022,True,0.135379,0.473827,0.390794,0.131763,0.501745,0.366492,0.129332,0.507185,0.363483
2022-08-07_Giresunspor_Ad. Demirspor,2.30,3.49,2.94,Ad. Demirspor,2.20,3.40,2.87,2.35,3.40,2.80,2022-08-07,T1,3,2,A,Giresunspor,2.35,3.45,2.95,2.39,3.69,3.07,2.38,3.4,3.00,2.30,3.40,2.90,True,7,8,2022,False,0.338843,0.259740,0.401417,0.327485,0.274854,0.397661,0.337143,0.268571,0.394286
2022-08-07_Karagumruk_Alanyaspor,2.53,3.46,2.65,Alanyaspor,2.40,3.50,2.55,2.55,3.30,2.60,2022-08-07,T1,4,2,A,Karagumruk,2.55,3.45,2.65,2.60,3.66,2.73,2.55,3.4,2.70,2.55,3.40,2.62,True,7,8,2022,False,0.301775,0.284024,0.414201,0.307692,0.301775,0.390533,0.306358,0.294798,0.398844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-03-21_Gent_Genk,NaN,NaN,NaN,Genk,2.50,3.20,2.63,2.45,3.50,2.60,2012-03-21,B1,3,1,H,Gent,2.40,3.20,2.60,NaN,NaN,NaN,2.60,3.4,2.62,2.45,3.25,2.62,False,21,3,2012,False,0.315726,0.300120,0.384154,0.304094,0.286550,0.409357,0.317073,0.292683,0.390244
2012-03-21_Germinal_Kortrijk,NaN,NaN,NaN,Kortrijk,1.91,3.40,3.60,1.95,3.50,3.60,2012-03-21,B1,2,0,H,Germinal,1.80,3.30,3.80,NaN,NaN,NaN,1.91,3.5,3.90,1.95,3.40,3.50,False,21,3,2012,False,0.404040,0.214366,0.381594,0.397790,0.215470,0.386740,0.426966,0.202247,0.370787
2012-03-21_Cercle Brugge_Lokeren,NaN,NaN,NaN,Lokeren,2.20,3.25,3.00,2.10,3.30,3.30,2012-03-21,B1,1,0,H,Cercle Brugge,2.10,3.20,3.10,NaN,NaN,NaN,2.20,3.3,3.25,2.15,3.25,3.10,False,21,3,2012,False,0.355030,0.260355,0.384615,0.379310,0.241379,0.379310,0.369048,0.250000,0.380952
2012-03-21_Standard_Mechelen,NaN,NaN,NaN,Mechelen,2.63,3.20,2.50,2.65,3.30,2.50,2012-03-21,B1,1,2,A,Standard,2.50,3.20,2.50,NaN,NaN,NaN,2.70,3.3,2.60,2.60,3.25,2.50,False,21,3,2012,True,0.300120,0.315726,0.384154,0.295858,0.313609,0.390533,0.304878,0.304878,0.390244


In [1086]:
standad_attributes = ['Target','atHome','Date','Div']
bookmaker_attributes = get_bookmaker(bm = 'BW',df = df)+get_bookmaker(bm='B365',df = df)+get_bookmaker(bm='IW',df = df)
bet_on_team = get_bookmaker(bm='Team',df=df[bookmaker_attributes])
bet_on_opponent = get_bookmaker(bm='Opponent',df=df[bookmaker_attributes])
bet_on_draw = get_bookmaker(bm='Draw',df=df[bookmaker_attributes])


In [1087]:
bet = "BW"

df_derived_b365 = get_odd_pred(bet="B365",df=df)
df_derived_iw = get_odd_pred(bet="IW",df=df)
df_derived_bw = get_odd_pred(bet="BW",df=df)


In [1088]:
df_derived_team = derived_odds(sight = 'Team',df=df, odds=bet_on_team)
df_derived_opponent = derived_odds(sight = 'Opponent',df=df, odds=bet_on_opponent)
df_derived_draw = derived_odds(sight = 'Draw',df=df, odds=bet_on_draw)


# Modelldatensatz erstellen

In [1089]:
model_data = pd.concat([df[standad_attributes],df_derived_team,df_derived_opponent,df_derived_draw,df[bookmaker_attributes],df[["Div","Team"]]],axis=1)
#model_data = pd.concat([df[standad_attributes],df[bookmaker_attributes],df_derived_team,df_derived_opponent,df_derived_draw],axis=1)
model_data = pd.concat([df[standad_attributes],df[bookmaker_attributes],df[["Team","month","year"]],df_derived_team,df_derived_opponent,df_derived_draw,df_derived_b365,df_derived_iw,df_derived_bw],axis=1)


In [1090]:
model_data.replace([np.inf, -np.inf], np.nan, inplace=True)
model_data =  model_data.dropna(0)

In a future version of pandas all arguments of DataFrame.dropna will be keyword-only


In [1091]:
from sklearn import preprocessing
le_div = preprocessing.LabelEncoder()
le_team = preprocessing.LabelEncoder()

le_team.fit(list(set(model_data.Team)))
model_data.Team = le_team.transform(model_data.Team)



# array([2, 2, 1]...)
# >>> list(le.inverse_transform([2, 2, 1]))
# ['tokyo', 'tokyo', 'paris']

In [1092]:
le_team.fit(list(set(model_data.Team)))
model_data.Team = le_team.transform(model_data.Team)

In [1093]:
le_div.fit(list(set(model_data.Div)))
model_data.Div = le_div.transform(model_data.Div)


In [1094]:
# drop_list = ["Date","Team","Div",'BW_Opponent', 'BW_Draw', 'BW_Team',
#        'B365_Opponent', 'B365_Draw', 'B365_Team', 'IW_Opponent', 'IW_Draw',
#        'IW_Team', 'month', 'year', 'Max_Team', 'Min_Team', 'Avg_Team',
#        'Span_Team', 'Ratio_Team', 'Max_Opponent', 'Min_Opponent',
#        'Avg_Opponent', 'Span_Opponent', 'Ratio_Opponent', 'Max_Draw',
#        'Min_Draw', 'Avg_Draw', 'Span_Draw', 'Ratio_Draw']
# model_data = model_data.drop(columns=drop_list)

Aufteilen in Training, Test und Validierung

In [1095]:
import datetime
datum_1_1_2020 = datetime.datetime(2021, 6, 1)
anwendung = model_data[model_data.Date>datum_1_1_2020]
model_data = model_data[model_data.Date<datum_1_1_2020]


In [1096]:
model_data.columns
model_data.to_csv(GLOBALS.DATA_FOLDER + 'model_data.csv')
anwendung.to_csv(GLOBALS.DATA_FOLDER + 'val_data.csv')

# Modellieren

In [1079]:
from sklearn.model_selection import train_test_split

y = model_data['Target']
X = model_data.drop(['Target'], axis=1)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0, stratify=y)

In [1009]:


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV  
from xgboost import XGBClassifier, XGBRegressor
model = RandomForestClassifier(
      n_estimators=200, random_state=0, verbose=1, min_samples_leaf=5, min_samples_split=10)
#model = XGBClassifier(n_estimators=200,random_state = 0,verbose=1, min_samples_leaf= 5, min_samples_split = 10)
parameters = {
	'selector__threshold': [0],
	'classifier__n_neighbors': [1],
	'classifier__max_depth': [5],
	'classifier__learning_rate': [0.01],
	'classifier__n_estimators': [100,]
}
 
 
#model = GridSearchCV(XGBClassifier(), parameters,cv=10,scoring = 'precision').fit(X_train, y_train)
#model = gridsearch(model, X_train, y_train)
#model = XGBRegressor()
model.fit(X=X_train, y=y_train)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   48.3s finished


RandomForestClassifier(min_samples_leaf=5, min_samples_split=10,
                       n_estimators=200, random_state=0, verbose=1)

In [1010]:
y_anw = anwendung['Target']
X_anw  = anwendung.drop(['Target'], axis=1)

evaluate_model(model, y_anw, X_anw, show=False,threshold=0.5)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.7s finished


AUC Score :  0.61
F1 Score :  0.447
Accuracy Score :  0.677
Precision Score :  0.608
Recall Score :  0.353
Confusion Matrix : /n[[8809 1359]
 [3850 2104]]
----------------------------------------------------------------------------------------------------
Feature Importance
Important
BW_Team_odd_pred          0.153132
B365_Team_odd_pred        0.139419
BW_opponent_odd_pred      0.118025
IW_Team_odd_pred          0.114507
B365_opponent_odd_pred    0.112914
IW_opponent_odd_pred      0.097722
BW_Team_draw_pred         0.089056
IW_Team_draw_pred         0.081172
B365_Team_draw_pred       0.079555
atHome                    0.014497
dtype: float64
Not Important
atHome                    0.014497
B365_Team_draw_pred       0.079555
IW_Team_draw_pred         0.081172
BW_Team_draw_pred         0.089056
IW_opponent_odd_pred      0.097722
B365_opponent_odd_pred    0.112914
IW_Team_odd_pred          0.114507
BW_opponent_odd_pred      0.118025
B365_Team_odd_pred        0.139419
BW_Team_odd_pred     

{'roc_auc_score': 0.61,
 'f1_score': 0.447,
 'accuracy_score': 0.677,
 'precision_score': 0.608,
 'recall_score': 0.353}

In [1011]:

evaluate_model(model, y_test, X_test, show=False,threshold=0.5)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC Score :  0.605
F1 Score :  0.443
Accuracy Score :  0.671
Precision Score :  0.584
Recall Score :  0.357
Confusion Matrix : /n[[17476  3020]
 [ 7619  4233]]
----------------------------------------------------------------------------------------------------
Feature Importance
Important
BW_Team_odd_pred          0.153132
B365_Team_odd_pred        0.139419
BW_opponent_odd_pred      0.118025
IW_Team_odd_pred          0.114507
B365_opponent_odd_pred    0.112914
IW_opponent_odd_pred      0.097722
BW_Team_draw_pred         0.089056
IW_Team_draw_pred         0.081172
B365_Team_draw_pred       0.079555
atHome                    0.014497
dtype: float64
Not Important
atHome                    0.014497
B365_Team_draw_pred       0.079555
IW_Team_draw_pred         0.081172
BW_Team_draw_pred         0.089056
IW_opponent_odd_pred      0.097722
B365_opponent_odd_pred    0.112914
IW_Team_odd_pred          0.114507
BW_opponent_odd_pred      0.118025
B365_Team_odd_pred        0.139419
BW_Team_odd_pred

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.8s finished


{'roc_auc_score': 0.605,
 'f1_score': 0.443,
 'accuracy_score': 0.671,
 'precision_score': 0.584,
 'recall_score': 0.357}

In [1012]:
model_data.dtypes

Target                       bool
atHome                       bool
B365_Team_odd_pred        float64
B365_opponent_odd_pred    float64
B365_Team_draw_pred       float64
IW_Team_odd_pred          float64
IW_opponent_odd_pred      float64
IW_Team_draw_pred         float64
BW_Team_odd_pred          float64
BW_opponent_odd_pred      float64
BW_Team_draw_pred         float64
dtype: object

In [1013]:
y_pred_proba = model.predict_proba(X_anw)[:,1]
y_pred_proba = pd.Series(y_pred_proba,index=X_anw.index,name='pred')
results = pd.concat([y_pred_proba,y_anw,df.FTG_Team,df.FTG_Opponent,df.BW_Team,df.BW_Opponent,df.BW_Draw,df.IW_Team,df.IW_Opponent,df.IW_Draw,df.B365_Team,df.B365_Opponent,df.B365_Draw,df.Date,df.year,df.month,df.day],axis=1,join='inner')
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.9s finished


,pred,Target,FTG_Team,FTG_Opponent,BW_Team,BW_Opponent,BW_Draw,IW_Team,IW_Opponent,IW_Draw,B365_Team,B365_Opponent,B365_Draw,Date,year,month,day
2022-08-05_Istanbulspor_Trabzonspor,0.173436,False,0,2,4.60,1.70,3.75,4.80,1.70,3.85,4.70,1.61,3.75,2022-08-05,2022,8,5
2022-08-06_Sivasspor_Gaziantep,0.583549,False,1,1,2.00,3.50,3.50,2.05,3.50,3.55,1.95,3.30,3.50,2022-08-06,2022,8,6
2022-08-06_Besiktas_Kayserispor,0.799641,True,1,0,1.51,5.75,4.20,1.53,6.00,4.30,1.50,5.25,4.33,2022-08-06,2022,8,6
2022-08-07_Giresunspor_Ad. Demirspor,0.300997,False,2,3,2.80,2.35,3.40,2.95,2.35,3.45,2.87,2.20,3.40,2022-08-07,2022,8,7
2022-08-07_Karagumruk_Alanyaspor,0.411834,False,2,4,2.60,2.55,3.30,2.65,2.55,3.45,2.55,2.40,3.50,2022-08-07,2022,8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-10_Oud-Heverlee Leuven_Gent,0.113397,False,0,5,12.00,1.23,6.50,10.00,1.25,6.50,9.50,1.22,6.00,2022-04-10,2022,4,10
2022-04-10_Anderlecht_Kortrijk,0.834297,True,3,2,1.42,7.25,4.80,1.40,7.00,5.00,1.33,7.00,4.75,2022-04-10,2022,4,10
2022-04-10_Genk_Seraing,0.754862,True,2,0,1.49,6.25,4.50,1.50,6.00,4.50,1.50,5.75,4.20,2022-04-10,2022,4,10
2022-04-10_Beerschot VA_St. Gilloise,0.059524,False,0,5,17.50,1.17,7.50,15.00,1.17,7.50,12.00,1.14,7.50,2022-04-10,2022,4,10


In [1014]:
stichtage =results[["year","month","day"]].drop_duplicates()
stichtage

,year,month,day
2022-08-05_Istanbulspor_Trabzonspor,2022,8,5
2022-08-06_Sivasspor_Gaziantep,2022,8,6
2022-08-07_Giresunspor_Ad. Demirspor,2022,8,7
2022-08-08_Buyuksehyr_Kasimpasa,2022,8,8
2022-08-12_Trabzonspor_Hatayspor,2022,8,12
...,...,...,...
2022-07-24_Bielefeld_Regensburg,2022,7,24
2021-07-23_Schalke 04_Hamburg,2021,7,23
2021-07-25_Nurnberg_Erzgebirge Aue,2021,7,25
2021-07-30_Karlsruhe_Darmstadt,2021,7,30


In [1015]:
r = evaluate_my_bets(bet='BW',pred_min=pred_min,odd_min=odd_min,results = match_day,method='kombi')
r>0

Boolean Series key will be reindexed to match DataFrame index.


False

In [1016]:
i=20
year = stichtage.year.iloc[i]
month = stichtage.month.iloc[i]
day =stichtage.day.iloc[i]
match_day = results[(results.year == year)&(results.month == month)&(results.day == day)]
ren = []
pred_min = 0.6
odd_min = 1.8
for i in range(0,len(stichtage)):
    year = stichtage.year.iloc[i]
    month = stichtage.month.iloc[i]
    day =stichtage.day.iloc[i]
    match_day = results[(results.year == year)&(results.month == month)&(results.day == day)]
    r = evaluate_my_bets(bet='BW',pred_min=pred_min,odd_min=odd_min,results = match_day,method='kombi')
    if r>0:
        print(i)
        ren +=[r]
        #ren +=[match_day.Target[(match_day.pred>pred_min)&(match_day.BW_Team>odd_min)]]

sum(ren)

Boolean Series key will be reindexed to match DataFrame index.


0
3
5
6
11
24
26
34
41
43
48
49
53
56
60
63
64
68
85
87
91
104
110
111
116
126
130
133
135
147
153
160
178
190
211
228
234
241
244
253
256
264


108.98138000000003

In [1017]:
sum(ren)/len(stichtage)

0.3427087421383649

In [1018]:
bet = 'BW'
pred_min = 0.6
odd_min = 2.1
def evaluate_my_bets(bet,pred_min,odd_min,results,method = 'single'):
    pred_bet = results[bet+"_Team"]/((results[bet+"_Team"]+results[bet+"_Opponent"]+results[bet+"_Draw"]))

    kandidaten = results[results.pred>pred_bet][results.pred>pred_min][results[bet+"_Team"]>odd_min]

    kandidaten = kandidaten[["Target","FTG_Team","FTG_Opponent",bet+"_Team","pred"]].join(df["Div"])

    
    if kandidaten.shape[0]>0:
        if method == 'single':
            rendite = sum(kandidaten[df.Target][bet+"_Team"])/kandidaten.shape[0]
        elif method == 'kombi':
            bet_on = (match_day.pred>pred_min)&((match_day[bet+"_Team"]>odd_min))
            bets = match_day.Target[bet_on]
            if np.prod(bets) == 1:
                rendite = np.prod(match_day[bet+"_Team"][bet_on])
            else: 
                rendite = 0
    else:
        rendite = 0
    # print("getippte Spiele",kandidaten.shape[0])
    # print("Richtige Spiele:",sum(kandidaten.Target))
    # print("Rendite:",rendite)
    kandidaten
    return rendite

evaluate_my_bets(bet='IW',pred_min=0.7,odd_min=1,results = match_day,method = 'kombi')

Boolean Series key will be reindexed to match DataFrame index.


0

In [1019]:
            bet_on = (match_day.pred>0.4)&((match_day[bet+"_Team"]>odd_min))
            bets = match_day.Target[bet_on]
            bets

2021-07-27_Beerschot VA_Cercle Brugge    False
2021-07-27_Cercle Brugge_Beerschot VA     True
Name: Target, dtype: bool

In [1020]:
match_day
bets = match_day.Target[match_day.pred>pred_min]
print(np.prod(bets))
if np.prod(bets) == 1:
    rendite = np.prod(match_day[bet+"_Team"][match_day.pred>pred_min])
rendite

#match_day[match_day.Target==(match_day.pred>0.5)][match_day.Target].BW_Team

1


1.0

In [1021]:
match_day

,pred,Target,FTG_Team,FTG_Opponent,BW_Team,BW_Opponent,BW_Draw,IW_Team,IW_Opponent,IW_Draw,B365_Team,B365_Opponent,B365_Draw,Date,year,month,day
2021-07-27_Beerschot VA_Cercle Brugge,0.432258,False,0,1,2.2,3.2,3.5,2.25,2.95,3.35,2.2,3.1,3.6,2021-07-27,2021,7,27
2021-07-27_Cercle Brugge_Beerschot VA,0.475266,True,1,0,3.2,2.2,3.5,2.95,2.25,3.35,3.1,2.2,3.6,2021-07-27,2021,7,27


In [1022]:
evaluate_my_bets(bet='BW',pred_min=0.7,odd_min=2.0,results = results[results.year==2021])
evaluate_my_bets(bet='BW',pred_min=0.7,odd_min=2.,results = results[results.year==2022])

Boolean Series key will be reindexed to match DataFrame index.


0

In [1023]:
umsatz  =[]
for p in range(50,100):
    p = p/100
    umsatz += [evaluate_my_bets(bet='IW',pred_min=p,odd_min=2,results = results[results.year==2019])]


max(umsatz)


0

In [1024]:
evaluate_my_bets(bet='IW',pred_min=0.6,odd_min=2,results = results[results.year==2019])

0